# Tree-of-Thoughts Interactive Notebook

This notebook is an interactive version of the [Tree-of-Thoughts](https://github.com/zbambergerNLP/tree-of-thoughts) library.
It allows you to explore [Tree-of-Thoughts](https://arxiv.org/pdf/2305.10601) functionality to create persuasive arguments and engage in debates.

![Tree-of-Thoughts](figures/tot_diagram.png)

## How Tree-of-Thoughts Works

Tree-of-Thoughts is a sequential prompting framework that is meant to evoke complex reasoning and planning capabilities in language models. We apply the Tree-of-Thoughts framework to the task of generating persuasive arguments (rather than tasks like `game-of-24`, `crosswords` or `creative writing`, which were used in the original paper).

A core idea in Tree-of-Thoughts is iterative expansion of a tree structure. In this tree structure, nodes represent "thoughts" ($z_t$), which are textual responses that incorporate intermediate/reasoning steps towards a final response. As part of the tree structure, a "thought" in layer `t` creates a set of "child thoughts" in layer `t+1`. This is known as the "branching" step (by a component known in the original paper as "Thought Generator", $G()$). 

Since unmediated branching can lead to a combinatorial explosion, we use a "debate judge" to evaluate the quality of each child thought after each expansion. The "debate judge" is a model that scores the quality of a thought based on its quality (a component known in the original paper as "States Evaluator", $V()$). Only the top scoring nodes in layer `t+1` are retained for further expansion. This is known as the "selection" step. 

See the full psuedocode excerpt from [Tree-of-Thoughts](https://arxiv.org/pdf/2305.10601) which describes the process:

![Tree-of-Thoughts Pseudocode](figures/tot_bfs_pseudocode.png)

### Our Interpretation

We interpret the Tree-of-Thoughts framework as a way to generate persuasive arguments. In this notebook we use closed-source language models (`gpt-3.5-turbo`, `gpt-4o`, `gpt-4o-mini`) to serve as both the "Thought Generator" and the "States Evaluator".

Rather than viewing nodes in the tree as "thoughts", we view them as "argument states" (which can include context about both the existing conversation and previous drafts). The "thoughts" in the original paper are meant to be intermediate steps towards a final response. In our case, the "arguments" are meant to be intermediate drafts towards a final persuasive argument.

### Alternative Interpretations (and why we chose ours instead)

While we chose to iterate over drafts at each layer of the tree, it is also possible to consider alternative interpretations of the Tree-of-Thoughts framework:

* One could consider each layer of the tree as an "expansion" of the argument (i.e., a child node presents an argument that is more detailed than its parent). We chose not to go with this approach because we didn't want the depth of a tree to be directly proportional to the length of the output.

* Another alternative interpretation is to consider pre-defined "instructions" for each layer of the tree (e.g., the first layer is meant for planning, the second layer is meant for identifying relevant facts, etc...). However, this approach would require a lot of manual work to define the instructions for each layer, and is not necessarily generalizable or scalable.

We therefore chose to iterate over drafts at each layer of the tree, as it is a simple and generalizable approach that can be applied to a wide range of argument generation tasks.

## Setup

In [2]:
# Installation
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Initialization

# Interactively ask for the user's input
openai_key = input("Please enter your OpenAI API key: ")
model_name_index = input("""
Please enter the index which corresponds with the name of the language model you wish to use. 
Your options are:
[1] gpt-3.5-turbo
[2] gpt-4o
[3] gpt-4o-mini
""".strip())
model_name = {
    "1": "gpt-3.5-turbo",
    "2": "gpt-4o",
    "3": "gpt-4o-mini"
}[model_name_index]
max_tokens = int(input("Please enter the maximum number of tokens you would like to generate: "))


In [5]:

from utils import set_up_dspy

set_up_dspy(
    openai_key=openai_key,
    model_name=model_name,
    max_tokens=max_tokens,
)

## Initializing Tree-of-Thoughts

### Creating a `State` instance

A `State` (our framework's equivalent to a "thought") is a representation of the current state of a debate. It contains the following information:

- `topic`: The topic of the argumentative conversation. For example, "The US government should increase the national minimum wage."
- `stance`: The stance of the debator towards the topic. Either 'PRO' or 'ANTI'. 
- `conversation`: The argumentative conversation. A list of strings, each representing a single persuasive argument. Note that 
    the stance of message `i` is opposite in stance to message `i+1`. 

**NOTE**: If you want the LLM-debator (i.e., the model that underlies the Tree-of-Thoughts) to generate an argument **without** a conversation as context (i.e., a single argument), set the `conversation` to `[]`.

We provide you with a default 'State' object below. Feel free to modify it as you see fit. 

In [9]:
from tree import State

# Define the initial state
initial_state = State(
    topic="The US government should increase the national minimum wage",
    stance="PRO",
    conversation=[
        "The US government is in a state of huge and nearly unpayable debt. Increasing the national minimum wage would only exacerbate this issue.",      
    ],
)
initial_state

State(topic='The US government should increase the national minimum wage', stance='PRO', conversation=['The US government is in a state of huge and nearly unpayable debt. Increasing the national minimum wage would only exacerbate this issue.'])

### Additional Abstractions

**State**: 
- `State`: Represents the current state of the debate. Contains the topic, stance, and conversation.
- `DraftState`: Extends `State` and contains additional information about previous drafts (given the same topic, stance, and conversation). The most recent draft was created by the parent node.

**Node**:
- `Node`: Wraps a `State` object. Contains additional information about the node's parent, children, the judge's score for the node, and the reasoning behind the score.
- `TreeOfThoughtsNode`: Wraps a `DraftState` object. Contains additional information about the node's parent, children, the judge's score for the node, and the reasoning behind the score.

**Edge**:
- `Edge`: Represents a connection between two nodes. Contains the new argument (produced by the node of the source of the edge) and the reasoning behind that argument.

**Tree**:
- `Tree`: Represents a collection of nodes and edges that form a 'Tree of Thoughts'. Contains a dedicated field that points to the root node.

### Creating a 'Tree-of-Thoughts' instance

The Tree-of-Thoughts module utilizes two key components:

1. `draft_predictor` ("Thought Generator"): A that creates a draft argument given the current state of the debate. 
2. `debate_judge` ("States Evaluator"): A module that evaluates the quality of an argument (either in the context of a conversation, or in isolation).

You can decide whether or not to initialize these parameters with `chain_of_thoughts`, which on the one hand provides reasoning for outputs and tends to improve quality, but on the other hand increases computation time and cost.

Moreover, you can specify the type of `debate_judge` you want to use (either `score` or `vote`). The `score` method evaluates each new argument in isolation, and provides a score. The `vote` method evaluates all of the new arguments at once, and then converts votes into scores.

Finally, by specifying the `node_selection_strategy`, you can decide how the Tree-of-Thoughts selects the next argument to generate. The `random` strategy selects a collection of weighted random arguments, while the `best` strategy selects the arguments



In [13]:
from tree_of_thoughts import TreeOfThoughts

tot_module = TreeOfThoughts(
    use_chain_of_thought=True,
    node_selection_strategy="greedy",
    evaluation_strategy="score",
)

## Running Tree-of-Thoughts

When running tree of thoughts, you can specify the following parameters:

- `depth`: The maximum depth (number of layers) of the tree. Increasing the depth improves the nuances of generated arguments, but also increases computation time and cost.
- `top_k`: The number of top arguments to select at each layer. Increasing the top_k arguments expands the search space, which can improve the quality of the generated arguments. However, it also increases computation time and cost.
- `generation_temperature`: The temperature to use when generating arguments. A higher temperature results in more diverse arguments, while a lower temperature results in more conservative arguments.
- `judge_temperature`: The temperature to use when evaluating arguments. A higher temperature results in more diverse evaluations, while a lower temperature results in more conservative evaluations.
- `n_samples_generation`: The number of samples to generate for each argument. This parameter also increases the search space, but takes place before nodes are scored by the judge. That is, while `top_k` specifies the amount of judged arguments to keep, `n_samples_generation` specifies the amount of arguments to generate before judging.


In [ ]:
response = tot_module(
    state=initial_state,
    depth=2,
    top_k=2,
    generation_temperature=0.7,
    judge_temperature=0.7,
    n_samples_generation=3,
    n_samples_judge=5,
    response_length="a few sentences",
)
print(f'\n\nThe response is:\n\n{response}')